# CPSC 4820 Project - Fire Intensity Prediction using Machine Learning

In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Descriptive Analysis

In [14]:
hotspots = pd.read_csv("data/combined_hotspots.csv")

In [15]:
hotspots.head()

,lat,lon,rep_date,source,sensor,satellite,agency,temp,rh,ws,...,greenup,elev,sfl,cfl,tfc0,ecozone,sfc0,cbh,uid,fid
0,53.0170,-124.689,2019/03/27 21:15:00.000,NASA,MODIS,Aqua,BC,3.761,59,11.673,...,0,1178,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN
1,50.5910,-119.904,2019/11/17 20:29:00.000,USFS,IBAND,JPSS1,BC,7.732,84,10.635,...,0,761,NaN,2.1,0.140000,14.0,0.140000,NaN,NaN,NaN
2,53.1846,-124.001,2019/03/27 19:42:00.000,NASA,IBAND,S-NPP,BC,2.090,64,11.087,...,0,1291,NaN,2.7,0.095046,NaN,0.095046,NaN,NaN,NaN
3,54.0710,-123.757,2019/11/17 21:18:00.000,USFS,IBAND,N,BC,8.218,61,14.859,...,0,1266,NaN,0.6,0.040000,14.0,0.040000,NaN,NaN,NaN
4,52.0966,-122.069,2019/03/29 11:00:00.000,NASA,IBAND,S-NPP,BC,11.225,31,7.106,...,0,734,NaN,2.2,0.316152,NaN,0.316152,NaN,NaN,NaN


In [16]:
hotspots.shape

(1033890, 37)

In [17]:
hotspots.columns

Index(['lat', 'lon', 'rep_date', 'source', 'sensor', 'satellite', 'agency',
       'temp', 'rh', 'ws', 'wd', 'pcp', 'ffmc', 'dmc', 'dc', 'isi', 'bui',
       'fwi', 'fuel', 'ros', 'sfc', 'tfc', 'bfc', 'hfi', 'cfb', 'estarea',
       'pcuring', 'greenup', 'elev', 'sfl', 'cfl', 'tfc0', 'ecozone', 'sfc0',
       'cbh', 'uid', 'fid'],
      dtype='object')

In [18]:
hotspots.dtypes

lat          float64
lon          float64
rep_date      object
source        object
sensor        object
satellite     object
agency        object
temp         float64
rh             int64
ws           float64
wd             int64
pcp          float64
ffmc         float64
dmc          float64
dc           float64
isi          float64
bui          float64
fwi          float64
fuel          object
ros          float64
sfc          float64
tfc          float64
bfc          float64
hfi          float64
cfb          float64
estarea      float64
pcuring      float64
greenup       object
elev           int64
sfl          float64
cfl          float64
tfc0         float64
ecozone      float64
sfc0         float64
cbh          float64
uid          float64
fid          float64
dtype: object

In [19]:
# Assessing features with missing values
null_values = hotspots.isnull().sum()
print("Columns with null values:\n", null_values[null_values > 0])

Columns with null values:
 source        696595
satellite      50110
ros              150
sfc              150
tfc              150
bfc           749015
hfi              150
cfb              150
estarea      1019088
pcuring       694831
greenup       694831
sfl             3541
tfc0             170
ecozone          237
sfc0             258
cbh           700136
uid          1021169
fid           337295
dtype: int64


In [20]:
# Calculate the percentage of missing values in each column
missing_percentage = hotspots.isnull().mean() * 100

# Convert the Series to a DataFrame for better readability
missing_percentage_df = missing_percentage.reset_index()
missing_percentage_df.columns = ['Column', 'Missing Percentage']
missing_percentage_df

,Column,Missing Percentage
0,lat,0.000000
1,lon,0.000000
2,rep_date,0.000000
3,source,67.376123
4,sensor,0.000000
5,satellite,4.846744
6,agency,0.000000
7,temp,0.000000
8,rh,0.000000
9,ws,0.000000


In [21]:
#Fill up the missing satelite information with 'Unknown'
hotspots['satellite'].fillna('unknown', inplace=True)

In [22]:
# Identify rows with missing values in the 'sfc' column
sfc_missing_indices = hotspots[hotspots['sfc'].isnull()].index

In [23]:
# Compare with other columns
columns_to_check = ['tfc', 'hfi', 'cfb', 'tfc0', 'sfc0']
missing_in_all = hotspots.loc[sfc_missing_indices, columns_to_check].isnull().all(axis=1)

# Check if all values are True (indicating the same rows are missing in all columns)
if missing_in_all.all():
    print("The rows with missing values in 'sfc' are the same as those in 'tfc', 'hfi', 'cfb', 'tfc0', and 'sfc0'.")
else:
    print("The rows with missing values in 'sfc' are NOT the same as those in 'tfc', 'hfi', 'cfb', 'tfc0', and 'sfc0'.")

# Display the indices where the missing values do not match
mismatched_indices = missing_in_all[~missing_in_all].index
print("Indices with mismatched missing values:\n", mismatched_indices)

The rows with missing values in 'sfc' are the same as those in 'tfc', 'hfi', 'cfb', 'tfc0', and 'sfc0'.
Indices with mismatched missing values:
 Int64Index([], dtype='int64')


In [27]:
#hotspots = hotspots.drop(sfc_missing_indices)
hotspots = hotspots.drop(columns=['cbh', 'bfc','pcuring', 'greenup','source','estarea','fid','uid'])
hotspots = hotspots.dropna(subset=['sfl','tfc0','ecozone','sfc0'])

In [28]:
# Check for duplicate rows
num_duplicates = hotspots.duplicated().sum()
hotspots = hotspots.drop_duplicates()

In [29]:
hotspots = hotspots[hotspots['hfi'] != 0]

In [30]:
hotspots = hotspots.drop(columns=['tfc0', 'sfc0'])
hotspots = hotspots.drop(columns=['agency'])

In [31]:
hotspots['rep_date'] = pd.to_datetime(hotspots['rep_date'], errors='coerce')

In [32]:
hotspots['year'] = hotspots['rep_date'].dt.year
hotspots['month'] = hotspots['rep_date'].dt.month
hotspots['day'] = hotspots['rep_date'].dt.dayofyear

In [ ]:
hotspots.to_csv('data/cleaned_hotspots', index=False)

### Sampling

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
print(hotspots['year'].value_counts())

2023    680106
2021    229679
2022     47713
2019     14094
2020      8234
Name: year, dtype: int64


In [36]:
years_of_interest = [2019, 2020, 2021, 2022, 2023]
hotspots = hotspots[hotspots['year'].isin(years_of_interest)]

# Perform stratified sampling to get a representative sample
sampled_df, _ = train_test_split(hotspots, test_size=0.25, stratify=hotspots['year'], random_state=1)

# Display the shape of the sampled DataFrame to verify
print(f"Sampled data shape: {sampled_df.shape}")

# Ensure each year is represented in the sampled data
print("Sampled data year distribution:")
print(sampled_df['year'].value_counts())

Sampled data shape: (734869, 29)
Sampled data year distribution:
2023    510079
2021    172259
2022     35785
2019     10570
2020      6176
Name: year, dtype: int64


In [37]:
def categorize_intensity(hfi):
    if hfi <= 10:
        return 'Low'
    elif hfi <= 100:
        return 'Moderate'
    else:
        return 'High'

# Apply the function to create the new 'Intensity' feature
sampled_df['Intensity'] = sampled_df['hfi'].apply(categorize_intensity)

# Display the first few rows to verify
sampled_df[['hfi', 'Intensity']].head()

,hfi,Intensity
86081,6080.0,High
477065,4420.0,High
657036,2728.0,High
548634,46.0,Moderate
553508,1486.0,High


In [38]:
sampled_df = sampled_df[sampled_df['hfi'] <= 60000]

# Verify the filtering step
print(f"Number of rows after filtering 'hfi' > 60000: {sampled_df.shape[0]}")

Number of rows after filtering 'hfi' > 60000: 733476


### EDA